# Day 4: Kalman Filter for State Estimation in Finance

## Week 20 - Bayesian Methods

---

### Learning Objectives
- Understand Kalman Filter as a Bayesian recursive estimator
- Implement Kalman Filter from scratch for financial applications
- Apply Kalman filtering for trend extraction in price series
- Estimate time-varying hedge ratios (beta) for pairs trading
- Compare Kalman Filter with traditional smoothing methods

### Key Concepts
- **State Space Models**: Hidden state variables + noisy observations
- **Predict-Update Cycle**: Two-step recursive Bayesian inference
- **Kalman Gain**: Optimal weighting between prediction and observation
- **Dynamic Parameter Estimation**: Time-varying regression coefficients

---

### Why Kalman Filters in Finance?

1. **Trend Extraction**: Separate signal from noise in price data
2. **Pairs Trading**: Dynamic hedge ratio estimation
3. **Volatility Filtering**: Real-time volatility estimation
4. **Factor Models**: Time-varying factor loadings
5. **Market Making**: Estimate fair value from noisy quotes

## 1. Import Required Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
from typing import Tuple, Optional, List
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11

# Set random seed for reproducibility
np.random.seed(42)

print("Libraries imported successfully!")
print(f"NumPy version: {np.__version__}")
print(f"Pandas version: {pd.__version__}")

## 2. Generate Synthetic Financial Data with Hidden State

We'll create synthetic data where:
- **Hidden State**: The "true" underlying price/trend we want to estimate
- **Observations**: Noisy measurements of the hidden state

This setup mimics real financial markets where the "fair value" is unknown and we only observe noisy prices.

In [ ]:
def generate_synthetic_data(
    n_steps: int = 500,
    initial_state: float = 100.0,
    process_noise_std: float = 0.5,
    observation_noise_std: float = 2.0,
    drift: float = 0.02
) -> Tuple[np.ndarray, np.ndarray]:
    """
    Generate synthetic financial data with hidden state.
    
    Model:
    - Hidden state (true price): x_t = x_{t-1} + drift + process_noise
    - Observation (noisy price): y_t = x_t + observation_noise
    
    Parameters:
    -----------
    n_steps : int
        Number of time steps
    initial_state : float
        Starting value for hidden state
    process_noise_std : float
        Standard deviation of process noise (state evolution)
    observation_noise_std : float
        Standard deviation of observation noise
    drift : float
        Constant drift term (like expected return)
        
    Returns:
    --------
    true_states : np.ndarray
        The actual hidden states (ground truth)
    observations : np.ndarray
        Noisy observations
    """
    true_states = np.zeros(n_steps)
    observations = np.zeros(n_steps)
    
    # Initialize
    true_states[0] = initial_state
    observations[0] = true_states[0] + np.random.normal(0, observation_noise_std)
    
    # Generate time series
    for t in range(1, n_steps):
        # State evolution: random walk with drift
        process_noise = np.random.normal(0, process_noise_std)
        true_states[t] = true_states[t-1] + drift + process_noise
        
        # Observation: state + noise
        observation_noise = np.random.normal(0, observation_noise_std)
        observations[t] = true_states[t] + observation_noise
    
    return true_states, observations

# Generate synthetic data
n_steps = 500
true_states, observations = generate_synthetic_data(
    n_steps=n_steps,
    initial_state=100.0,
    process_noise_std=0.5,
    observation_noise_std=2.0,
    drift=0.02
)

# Visualize the data
fig, axes = plt.subplots(2, 1, figsize=(14, 8))

# Plot 1: True state vs observations
axes[0].plot(true_states, 'b-', label='True Hidden State', linewidth=2)
axes[0].plot(observations, 'r.', alpha=0.5, markersize=3, label='Noisy Observations')
axes[0].set_xlabel('Time')
axes[0].set_ylabel('Price')
axes[0].set_title('Synthetic Financial Data: Hidden State vs Noisy Observations')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Plot 2: Observation noise distribution
noise = observations - true_states
axes[1].hist(noise, bins=50, density=True, alpha=0.7, color='green', edgecolor='black')
axes[1].axvline(noise.mean(), color='red', linestyle='--', label=f'Mean: {noise.mean():.3f}')
axes[1].axvline(noise.std(), color='orange', linestyle='--', label=f'Std: {noise.std():.3f}')
axes[1].set_xlabel('Observation Noise')
axes[1].set_ylabel('Density')
axes[1].set_title('Distribution of Observation Noise')
axes[1].legend()

plt.tight_layout()
plt.show()

print(f"True state range: [{true_states.min():.2f}, {true_states.max():.2f}]")
print(f"Observation noise - Mean: {noise.mean():.4f}, Std: {noise.std():.4f}")

## 3. Define Kalman Filter Equations

### State Space Model

The Kalman Filter operates on a **linear state space model**:

**State Equation (Process Model):**
$$x_t = F \cdot x_{t-1} + B \cdot u_t + w_t, \quad w_t \sim \mathcal{N}(0, Q)$$

**Observation Equation (Measurement Model):**
$$y_t = H \cdot x_t + v_t, \quad v_t \sim \mathcal{N}(0, R)$$

Where:
- $x_t$: Hidden state vector at time $t$
- $y_t$: Observation vector at time $t$
- $F$: State transition matrix
- $H$: Observation matrix
- $Q$: Process noise covariance
- $R$: Observation noise covariance
- $w_t$, $v_t$: Process and observation noise (Gaussian)

---

### Kalman Filter Algorithm

#### **Predict Step** (Time Update)
Propagate state estimate and uncertainty forward:

$$\hat{x}_{t|t-1} = F \cdot \hat{x}_{t-1|t-1}$$
$$P_{t|t-1} = F \cdot P_{t-1|t-1} \cdot F^T + Q$$

#### **Update Step** (Measurement Update)
Incorporate new observation:

**Innovation (Measurement Residual):**
$$\tilde{y}_t = y_t - H \cdot \hat{x}_{t|t-1}$$

**Innovation Covariance:**
$$S_t = H \cdot P_{t|t-1} \cdot H^T + R$$

**Kalman Gain:**
$$K_t = P_{t|t-1} \cdot H^T \cdot S_t^{-1}$$

**State Update:**
$$\hat{x}_{t|t} = \hat{x}_{t|t-1} + K_t \cdot \tilde{y}_t$$

**Covariance Update:**
$$P_{t|t} = (I - K_t \cdot H) \cdot P_{t|t-1}$$

---

### Intuition Behind Kalman Gain

The Kalman Gain $K_t$ balances trust between:
- **Model prediction** (based on state transition)
- **Measurement** (actual observation)

$$K_t = \frac{\text{Prediction Uncertainty}}{\text{Prediction Uncertainty} + \text{Measurement Uncertainty}}$$

- If $R$ is small (accurate measurements): $K_t \to 1$ → Trust observations more
- If $Q$ is small (stable process): $K_t \to 0$ → Trust predictions more

## 4. Implement Kalman Filter Class

Let's build a complete Kalman Filter implementation from scratch.

In [ ]:
class KalmanFilter:
    """
    A general-purpose Kalman Filter implementation for state estimation.
    
    State Space Model:
        x_t = F @ x_{t-1} + B @ u_t + w_t,  w_t ~ N(0, Q)
        y_t = H @ x_t + v_t,                 v_t ~ N(0, R)
    
    Parameters:
    -----------
    dim_state : int
        Dimension of state vector
    dim_obs : int
        Dimension of observation vector
    F : np.ndarray
        State transition matrix (dim_state x dim_state)
    H : np.ndarray
        Observation matrix (dim_obs x dim_state)
    Q : np.ndarray
        Process noise covariance (dim_state x dim_state)
    R : np.ndarray
        Observation noise covariance (dim_obs x dim_obs)
    x0 : np.ndarray, optional
        Initial state estimate
    P0 : np.ndarray, optional
        Initial state covariance
    """
    
    def __init__(
        self,
        dim_state: int,
        dim_obs: int,
        F: np.ndarray,
        H: np.ndarray,
        Q: np.ndarray,
        R: np.ndarray,
        x0: Optional[np.ndarray] = None,
        P0: Optional[np.ndarray] = None
    ):
        self.dim_state = dim_state
        self.dim_obs = dim_obs
        
        # Model matrices
        self.F = np.atleast_2d(F)  # State transition
        self.H = np.atleast_2d(H)  # Observation
        self.Q = np.atleast_2d(Q)  # Process noise covariance
        self.R = np.atleast_2d(R)  # Observation noise covariance
        
        # Initial conditions
        self.x = x0 if x0 is not None else np.zeros((dim_state, 1))
        self.P = P0 if P0 is not None else np.eye(dim_state) * 1000  # Large initial uncertainty
        
        # Storage for filtering results
        self.state_history = []
        self.covariance_history = []
        self.kalman_gain_history = []
        self.innovation_history = []
        
    def predict(self, u: Optional[np.ndarray] = None, B: Optional[np.ndarray] = None):
        """
        Predict step: Propagate state and covariance forward.
        
        x_{t|t-1} = F @ x_{t-1|t-1} + B @ u_t
        P_{t|t-1} = F @ P_{t-1|t-1} @ F.T + Q
        """
        # State prediction
        self.x = self.F @ self.x
        if u is not None and B is not None:
            self.x += B @ u
            
        # Covariance prediction
        self.P = self.F @ self.P @ self.F.T + self.Q
        
        return self.x.copy(), self.P.copy()
    
    def update(self, y: np.ndarray):
        """
        Update step: Incorporate new observation.
        
        K_t = P_{t|t-1} @ H.T @ (H @ P_{t|t-1} @ H.T + R)^{-1}
        x_{t|t} = x_{t|t-1} + K_t @ (y_t - H @ x_{t|t-1})
        P_{t|t} = (I - K_t @ H) @ P_{t|t-1}
        """
        y = np.atleast_2d(y).reshape(-1, 1)
        
        # Innovation (measurement residual)
        innovation = y - self.H @ self.x
        
        # Innovation covariance
        S = self.H @ self.P @ self.H.T + self.R
        
        # Kalman gain
        K = self.P @ self.H.T @ np.linalg.inv(S)
        
        # State update
        self.x = self.x + K @ innovation
        
        # Covariance update (Joseph form for numerical stability)
        I_KH = np.eye(self.dim_state) - K @ self.H
        self.P = I_KH @ self.P @ I_KH.T + K @ self.R @ K.T
        
        # Store results
        self.state_history.append(self.x.copy())
        self.covariance_history.append(self.P.copy())
        self.kalman_gain_history.append(K.copy())
        self.innovation_history.append(innovation.copy())
        
        return self.x.copy(), self.P.copy(), K.copy()
    
    def filter(self, observations: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
        """
        Run the Kalman filter on a sequence of observations.
        
        Parameters:
        -----------
        observations : np.ndarray
            Array of observations (n_steps x dim_obs)
            
        Returns:
        --------
        filtered_states : np.ndarray
            Filtered state estimates
        filtered_covariances : np.ndarray
            Filtered state covariances
        """
        n_steps = len(observations)
        filtered_states = np.zeros((n_steps, self.dim_state))
        filtered_covariances = np.zeros((n_steps, self.dim_state, self.dim_state))
        
        for t, obs in enumerate(observations):
            # Predict
            self.predict()
            
            # Update with observation
            self.update(obs)
            
            # Store results
            filtered_states[t] = self.x.flatten()
            filtered_covariances[t] = self.P
            
        return filtered_states, filtered_covariances
    
    def get_uncertainty(self) -> float:
        """Return current state uncertainty (sqrt of trace of P)."""
        return np.sqrt(np.trace(self.P))
    
    def reset(self, x0: Optional[np.ndarray] = None, P0: Optional[np.ndarray] = None):
        """Reset filter to initial conditions."""
        self.x = x0 if x0 is not None else np.zeros((self.dim_state, 1))
        self.P = P0 if P0 is not None else np.eye(self.dim_state) * 1000
        self.state_history = []
        self.covariance_history = []
        self.kalman_gain_history = []
        self.innovation_history = []

print("KalmanFilter class defined successfully!")

## 5. Apply Kalman Filter to Estimate Hidden State

Now let's apply our Kalman Filter to the synthetic data to estimate the hidden true state from noisy observations.

In [ ]:
# Set up Kalman Filter for our synthetic data
# Model: x_t = x_{t-1} + drift + w_t (random walk with drift)
#        y_t = x_t + v_t (direct observation with noise)

# For simplicity, we'll ignore the drift term (or absorb it into the filter)
# State dimension: 1 (just the price level)
# Observation dimension: 1 (observed noisy price)

dim_state = 1
dim_obs = 1

# State transition matrix (F): Identity for random walk
F = np.array([[1.0]])

# Observation matrix (H): Direct observation
H = np.array([[1.0]])

# Process noise covariance (Q): Variance of state evolution
# This should match our synthetic data generation
Q = np.array([[0.5**2]])  # process_noise_std^2

# Observation noise covariance (R): Variance of measurement noise
R = np.array([[2.0**2]])  # observation_noise_std^2

# Initial state and covariance
x0 = np.array([[observations[0]]])  # Start from first observation
P0 = np.array([[10.0]])  # Moderate initial uncertainty

# Create Kalman Filter
kf = KalmanFilter(
    dim_state=dim_state,
    dim_obs=dim_obs,
    F=F,
    H=H,
    Q=Q,
    R=R,
    x0=x0,
    P0=P0
)

# Run the filter
filtered_states, filtered_covariances = kf.filter(observations)

# Extract variances for confidence intervals
filtered_variances = filtered_covariances[:, 0, 0]
filtered_std = np.sqrt(filtered_variances)

print("Kalman Filter applied successfully!")
print(f"Number of time steps: {len(filtered_states)}")
print(f"Final state estimate: {filtered_states[-1, 0]:.4f}")
print(f"Final true state: {true_states[-1]:.4f}")
print(f"Final uncertainty (std): {filtered_std[-1]:.4f}")

## 6. Visualize Filtered vs True State

Let's visualize how well the Kalman Filter estimates the hidden state, including confidence bands.

In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(14, 12))

time = np.arange(n_steps)

# Plot 1: Comprehensive comparison
ax1 = axes[0]
ax1.plot(time, observations, 'r.', alpha=0.3, markersize=3, label='Noisy Observations')
ax1.plot(time, true_states, 'b-', linewidth=2, label='True Hidden State')
ax1.plot(time, filtered_states[:, 0], 'g-', linewidth=2, label='Kalman Filter Estimate')

# Confidence bands (±2σ)
upper_band = filtered_states[:, 0] + 2 * filtered_std
lower_band = filtered_states[:, 0] - 2 * filtered_std
ax1.fill_between(time, lower_band, upper_band, color='green', alpha=0.2, label='95% Confidence Band')

ax1.set_xlabel('Time')
ax1.set_ylabel('Price')
ax1.set_title('Kalman Filter State Estimation')
ax1.legend(loc='upper left')
ax1.grid(True, alpha=0.3)

# Plot 2: Estimation error
estimation_error = filtered_states[:, 0] - true_states
ax2 = axes[1]
ax2.plot(time, estimation_error, 'purple', linewidth=1, alpha=0.7, label='Estimation Error')
ax2.fill_between(time, -2*filtered_std, 2*filtered_std, color='gray', alpha=0.3, label='±2σ Bounds')
ax2.axhline(0, color='black', linestyle='--', linewidth=0.8)
ax2.set_xlabel('Time')
ax2.set_ylabel('Error')
ax2.set_title('Estimation Error (Kalman Estimate - True State)')
ax2.legend()
ax2.grid(True, alpha=0.3)

# Plot 3: Filter uncertainty over time
ax3 = axes[2]
ax3.plot(time, filtered_std, 'orange', linewidth=2, label='Filter Uncertainty (σ)')
ax3.axhline(filtered_std[-1], color='red', linestyle='--', 
            label=f'Steady State: {filtered_std[-1]:.4f}')
ax3.set_xlabel('Time')
ax3.set_ylabel('Uncertainty (σ)')
ax3.set_title('Kalman Filter Uncertainty Over Time')
ax3.legend()
ax3.grid(True, alpha=0.3)
ax3.set_ylim(0, max(filtered_std) * 1.2)

plt.tight_layout()
plt.show()

# Quantitative analysis
rmse_kalman = np.sqrt(np.mean(estimation_error**2))
rmse_naive = np.sqrt(np.mean((observations - true_states)**2))

print("\n" + "="*60)
print("PERFORMANCE METRICS")
print("="*60)
print(f"Kalman Filter RMSE: {rmse_kalman:.4f}")
print(f"Naive (observations) RMSE: {rmse_naive:.4f}")
print(f"Improvement: {(1 - rmse_kalman/rmse_naive)*100:.2f}%")
print(f"Steady-state uncertainty: {filtered_std[-1]:.4f}")

## 7. Kalman Filter for Trend Extraction

Now let's apply the Kalman Filter to real financial data to extract underlying trends. We'll compare with traditional moving average smoothing.

In [ ]:
# Download real financial data
ticker = "SPY"
start_date = "2023-01-01"
end_date = "2024-12-31"

print(f"Downloading {ticker} data...")
data = yf.download(ticker, start=start_date, end=end_date, progress=False)
prices = data['Close'].values.flatten()
dates = data.index

print(f"Downloaded {len(prices)} data points")
print(f"Date range: {dates[0].strftime('%Y-%m-%d')} to {dates[-1].strftime('%Y-%m-%d')}")
print(f"Price range: ${prices.min():.2f} - ${prices.max():.2f}")

In [ ]:
def kalman_trend_filter(
    prices: np.ndarray,
    process_noise: float = 1.0,
    observation_noise: float = 10.0
) -> Tuple[np.ndarray, np.ndarray]:
    """
    Apply Kalman Filter for trend extraction from price data.
    
    The model assumes:
    - State: underlying trend level (random walk)
    - Observation: price = trend + noise
    
    Parameters:
    -----------
    prices : np.ndarray
        Price observations
    process_noise : float
        Variance of process noise (controls smoothness)
    observation_noise : float
        Variance of observation noise
        
    Returns:
    --------
    trend : np.ndarray
        Estimated trend
    uncertainty : np.ndarray
        Uncertainty (std) of trend estimate
    """
    n = len(prices)
    
    # Model parameters
    F = np.array([[1.0]])  # Random walk
    H = np.array([[1.0]])  # Direct observation
    Q = np.array([[process_noise]])
    R = np.array([[observation_noise]])
    
    # Initialize
    x0 = np.array([[prices[0]]])
    P0 = np.array([[observation_noise]])
    
    kf = KalmanFilter(
        dim_state=1, dim_obs=1,
        F=F, H=H, Q=Q, R=R,
        x0=x0, P0=P0
    )
    
    # Run filter
    trend, cov = kf.filter(prices)
    uncertainty = np.sqrt(cov[:, 0, 0])
    
    return trend.flatten(), uncertainty

# Apply Kalman Filter with different smoothness levels
trend_smooth, unc_smooth = kalman_trend_filter(prices, process_noise=0.5, observation_noise=20.0)
trend_medium, unc_medium = kalman_trend_filter(prices, process_noise=2.0, observation_noise=10.0)
trend_responsive, unc_responsive = kalman_trend_filter(prices, process_noise=10.0, observation_noise=5.0)

# Calculate traditional moving averages for comparison
df = pd.DataFrame({'price': prices}, index=dates)
df['SMA_20'] = df['price'].rolling(window=20).mean()
df['SMA_50'] = df['price'].rolling(window=50).mean()
df['EMA_20'] = df['price'].ewm(span=20, adjust=False).mean()

print("Trend extraction complete!")

In [ ]:
# Visualize trend extraction results
fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# Plot 1: Kalman Filter with different parameters
ax1 = axes[0]
ax1.plot(dates, prices, 'gray', alpha=0.5, linewidth=0.8, label='Price')
ax1.plot(dates, trend_smooth, 'b-', linewidth=2, label='Kalman (Smooth)')
ax1.plot(dates, trend_medium, 'g-', linewidth=2, label='Kalman (Medium)')
ax1.plot(dates, trend_responsive, 'r-', linewidth=2, label='Kalman (Responsive)')

# Add confidence band for medium filter
ax1.fill_between(dates, trend_medium - 2*unc_medium, trend_medium + 2*unc_medium, 
                 color='green', alpha=0.1)

ax1.set_xlabel('Date')
ax1.set_ylabel('Price ($)')
ax1.set_title(f'{ticker} - Kalman Filter Trend Extraction (Different Smoothness Levels)')
ax1.legend(loc='upper left')
ax1.grid(True, alpha=0.3)

# Plot 2: Kalman vs Moving Averages
ax2 = axes[1]
ax2.plot(dates, prices, 'gray', alpha=0.5, linewidth=0.8, label='Price')
ax2.plot(dates, trend_medium, 'g-', linewidth=2, label='Kalman Filter')
ax2.plot(dates, df['SMA_20'].values, 'b--', linewidth=1.5, label='SMA(20)')
ax2.plot(dates, df['SMA_50'].values, 'r--', linewidth=1.5, label='SMA(50)')
ax2.plot(dates, df['EMA_20'].values, 'm--', linewidth=1.5, label='EMA(20)')

ax2.set_xlabel('Date')
ax2.set_ylabel('Price ($)')
ax2.set_title(f'{ticker} - Kalman Filter vs Traditional Moving Averages')
ax2.legend(loc='upper left')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Key differences
print("\n" + "="*60)
print("KALMAN FILTER VS MOVING AVERAGES")
print("="*60)
print("""
Key Advantages of Kalman Filter:
1. No fixed look-back window - uses all available data optimally
2. Provides uncertainty estimates (confidence bands)
3. Adapts automatically to changing volatility
4. Can incorporate additional state variables (trend, velocity, etc.)
5. Works from the first observation (no warm-up period needed)

Moving Average Limitations:
1. Fixed window size regardless of market conditions
2. Equal weighting within window (SMA) or fixed decay (EMA)
3. No uncertainty quantification
4. Requires tuning window size
""")

## 8. Dynamic Hedge Ratio Estimation Using Kalman Filter

One of the most powerful applications of Kalman Filtering in quantitative finance is **dynamic hedge ratio estimation** for pairs trading.

### The Problem
In pairs trading, we look for two cointegrated assets and trade the spread:
$$\text{Spread}_t = Y_t - \beta_t \cdot X_t$$

Traditional OLS gives us a **static** $\beta$, but in reality, the hedge ratio changes over time.

### Kalman Filter Solution
Model $\beta$ as a time-varying state that follows a random walk:
$$\beta_t = \beta_{t-1} + w_t, \quad w_t \sim \mathcal{N}(0, Q)$$
$$Y_t = \beta_t \cdot X_t + v_t, \quad v_t \sim \mathcal{N}(0, R)$$

The Kalman Filter estimates the time-varying $\beta_t$ optimally.

In [ ]:
# Download data for a correlated pair
# Using gold miners ETF (GDX) and gold ETF (GLD) as a classic pairs trading example
print("Downloading pairs trading data...")
ticker1 = "GLD"  # Gold ETF
ticker2 = "GDX"  # Gold Miners ETF

data1 = yf.download(ticker1, start="2022-01-01", end="2024-12-31", progress=False)
data2 = yf.download(ticker2, start="2022-01-01", end="2024-12-31", progress=False)

# Align indices
common_dates = data1.index.intersection(data2.index)
y = data2.loc[common_dates, 'Close'].values.flatten()  # GDX (dependent)
x = data1.loc[common_dates, 'Close'].values.flatten()  # GLD (independent)
pair_dates = common_dates

print(f"Loaded {len(y)} data points for pair {ticker1}/{ticker2}")
print(f"Date range: {pair_dates[0].strftime('%Y-%m-%d')} to {pair_dates[-1].strftime('%Y-%m-%d')}")

In [ ]:
class KalmanHedgeRatio:
    """
    Kalman Filter for dynamic hedge ratio estimation.
    
    Model:
    State: beta_t (hedge ratio), follows random walk
    Observation: y_t = beta_t * x_t + noise
    
    This is a time-varying regression: Y = beta * X
    where beta changes over time as a random walk.
    """
    
    def __init__(
        self,
        delta: float = 1e-4,
        observation_variance: float = 1.0
    ):
        """
        Parameters:
        -----------
        delta : float
            Process noise variance (controls how fast beta can change)
            Smaller delta = more stable beta
        observation_variance : float
            Observation noise variance (R)
        """
        self.delta = delta
        self.observation_variance = observation_variance
        self.beta = None
        self.P = None
        
        # Storage
        self.betas = []
        self.variances = []
        self.spreads = []
        self.spread_vars = []
        
    def fit(self, y: np.ndarray, x: np.ndarray) -> dict:
        """
        Run Kalman filter to estimate time-varying hedge ratio.
        
        Parameters:
        -----------
        y : np.ndarray
            Dependent variable (asset to hedge)
        x : np.ndarray
            Independent variable (hedging instrument)
            
        Returns:
        --------
        results : dict
            Dictionary with betas, spreads, and uncertainties
        """
        n = len(y)
        
        # Initialize state: beta
        self.beta = 0.0
        self.P = 1.0  # Initial uncertainty
        
        self.betas = []
        self.variances = []
        self.spreads = []
        self.spread_vars = []
        
        for t in range(n):
            # Current observation matrix H_t = x_t
            H = x[t]
            
            # ===== PREDICT STEP =====
            # State prediction: beta_t|t-1 = beta_t-1|t-1 (random walk)
            beta_pred = self.beta
            
            # Covariance prediction: P_t|t-1 = P_t-1|t-1 + Q
            P_pred = self.P + self.delta
            
            # ===== UPDATE STEP =====
            # Innovation
            innovation = y[t] - H * beta_pred
            
            # Innovation variance: S = H * P * H' + R
            S = H * P_pred * H + self.observation_variance
            
            # Kalman gain: K = P * H' * S^{-1}
            K = P_pred * H / S
            
            # State update
            self.beta = beta_pred + K * innovation
            
            # Covariance update
            self.P = (1 - K * H) * P_pred
            
            # Calculate spread and its variance
            spread = y[t] - self.beta * x[t]
            spread_var = self.P * x[t]**2 + self.observation_variance
            
            # Store results
            self.betas.append(self.beta)
            self.variances.append(self.P)
            self.spreads.append(spread)
            self.spread_vars.append(spread_var)
        
        return {
            'betas': np.array(self.betas),
            'beta_std': np.sqrt(np.array(self.variances)),
            'spreads': np.array(self.spreads),
            'spread_std': np.sqrt(np.array(self.spread_vars))
        }

# Apply Kalman Filter for hedge ratio estimation
kf_hedge = KalmanHedgeRatio(delta=1e-4, observation_variance=1.0)
results = kf_hedge.fit(y, x)

# Compare with rolling OLS
window = 60
rolling_betas = []
for i in range(len(y)):
    if i < window:
        rolling_betas.append(np.nan)
    else:
        y_roll = y[i-window:i]
        x_roll = x[i-window:i]
        beta_ols = np.cov(y_roll, x_roll)[0, 1] / np.var(x_roll)
        rolling_betas.append(beta_ols)

rolling_betas = np.array(rolling_betas)

# Static OLS for comparison
beta_static = np.cov(y, x)[0, 1] / np.var(x)

print("Hedge ratio estimation complete!")
print(f"Static OLS beta: {beta_static:.4f}")
print(f"Kalman final beta: {results['betas'][-1]:.4f}")
print(f"Kalman beta range: [{results['betas'].min():.4f}, {results['betas'].max():.4f}]")

In [ ]:
# Visualize hedge ratio results
fig, axes = plt.subplots(3, 1, figsize=(14, 12))

# Plot 1: Time-varying hedge ratio
ax1 = axes[0]
ax1.plot(pair_dates, results['betas'], 'b-', linewidth=2, label='Kalman Filter β')
ax1.plot(pair_dates, rolling_betas, 'r--', linewidth=1.5, label=f'Rolling OLS (window={window})')
ax1.axhline(beta_static, color='gray', linestyle=':', linewidth=2, label=f'Static OLS β={beta_static:.3f}')

# Confidence bands
upper = results['betas'] + 2 * results['beta_std']
lower = results['betas'] - 2 * results['beta_std']
ax1.fill_between(pair_dates, lower, upper, color='blue', alpha=0.2, label='95% CI')

ax1.set_xlabel('Date')
ax1.set_ylabel('Hedge Ratio (β)')
ax1.set_title(f'Dynamic Hedge Ratio: {ticker2} = β × {ticker1}')
ax1.legend(loc='upper right')
ax1.grid(True, alpha=0.3)

# Plot 2: Spreads comparison
static_spread = y - beta_static * x
kalman_spread = results['spreads']

ax2 = axes[1]
ax2.plot(pair_dates, static_spread, 'r-', alpha=0.7, linewidth=1, label='Static Spread')
ax2.plot(pair_dates, kalman_spread, 'b-', alpha=0.9, linewidth=1.5, label='Kalman Spread')
ax2.axhline(0, color='black', linestyle='--', linewidth=0.8)
ax2.set_xlabel('Date')
ax2.set_ylabel('Spread')
ax2.set_title('Pairs Trading Spread: Static vs Dynamic Hedge Ratio')
ax2.legend()
ax2.grid(True, alpha=0.3)

# Plot 3: Spread z-score for trading signals
kalman_zscore = (kalman_spread - np.mean(kalman_spread)) / np.std(kalman_spread)
static_zscore = (static_spread - np.mean(static_spread)) / np.std(static_spread)

ax3 = axes[2]
ax3.plot(pair_dates, kalman_zscore, 'b-', linewidth=1.5, label='Kalman Z-Score')
ax3.plot(pair_dates, static_zscore, 'r-', alpha=0.5, linewidth=1, label='Static Z-Score')
ax3.axhline(2, color='green', linestyle='--', label='Entry: ±2σ')
ax3.axhline(-2, color='green', linestyle='--')
ax3.axhline(0, color='black', linestyle='-', linewidth=0.5)
ax3.fill_between(pair_dates, -2, 2, color='gray', alpha=0.1)
ax3.set_xlabel('Date')
ax3.set_ylabel('Z-Score')
ax3.set_title('Trading Signal: Spread Z-Score')
ax3.legend(loc='upper right')
ax3.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Performance comparison
print("\n" + "="*60)
print("SPREAD STATISTICS")
print("="*60)
print(f"\nStatic Spread:")
print(f"  Mean: {static_spread.mean():.4f}")
print(f"  Std:  {static_spread.std():.4f}")
print(f"  Sharpe (mean-reversion): {abs(static_spread.mean())/static_spread.std():.4f}")

print(f"\nKalman Spread:")
print(f"  Mean: {kalman_spread.mean():.4f}")
print(f"  Std:  {kalman_spread.std():.4f}")
print(f"  Sharpe (mean-reversion): {abs(kalman_spread.mean())/kalman_spread.std():.4f}")

print(f"\nStd Ratio (Kalman/Static): {kalman_spread.std()/static_spread.std():.4f}")
print("(Lower ratio = more stable spread = better for trading)")

## 9. Online Parameter Updates and Adaptive Filtering

The Kalman Filter naturally performs online learning. Let's demonstrate how the filter adapts as new data arrives and discuss tuning of the noise parameters $Q$ and $R$.

In [ ]:
def online_kalman_demo(prices: np.ndarray, q_values: list) -> dict:
    """
    Demonstrate online Kalman filtering with different process noise values.
    Shows how the filter adapts to new data.
    """
    results = {}
    
    for q in q_values:
        # Initialize filter
        F = np.array([[1.0]])
        H = np.array([[1.0]])
        Q = np.array([[q]])
        R = np.array([[10.0]])  # Fixed observation noise
        
        x = np.array([[prices[0]]])
        P = np.array([[100.0]])
        
        estimates = []
        uncertainties = []
        gains = []
        
        for t, obs in enumerate(prices):
            # Predict
            x_pred = F @ x
            P_pred = F @ P @ F.T + Q
            
            # Update
            innovation = obs - (H @ x_pred)[0, 0]
            S = (H @ P_pred @ H.T + R)[0, 0]
            K = (P_pred @ H.T / S)[0, 0]
            
            x = x_pred + K * innovation * np.ones((1, 1))
            P = (1 - K * H[0, 0]) * P_pred
            
            estimates.append(x[0, 0])
            uncertainties.append(np.sqrt(P[0, 0]))
            gains.append(K)
        
        results[q] = {
            'estimates': np.array(estimates),
            'uncertainties': np.array(uncertainties),
            'gains': np.array(gains)
        }
    
    return results

# Run with different Q values
q_values = [0.1, 1.0, 10.0]
online_results = online_kalman_demo(prices[:200], q_values)

# Visualize the effect of Q on filtering
fig, axes = plt.subplots(3, 1, figsize=(14, 12))

time = np.arange(200)
colors = ['blue', 'green', 'red']

# Plot 1: State estimates with different Q
ax1 = axes[0]
ax1.plot(time, prices[:200], 'gray', alpha=0.5, linewidth=0.8, label='Price')
for i, q in enumerate(q_values):
    ax1.plot(time, online_results[q]['estimates'], colors[i], 
             linewidth=2, label=f'Q = {q}')
ax1.set_xlabel('Time')
ax1.set_ylabel('Price')
ax1.set_title('Effect of Process Noise (Q) on Kalman Filter Smoothness')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot 2: Filter uncertainty over time
ax2 = axes[1]
for i, q in enumerate(q_values):
    ax2.plot(time, online_results[q]['uncertainties'], colors[i], 
             linewidth=2, label=f'Q = {q}')
ax2.set_xlabel('Time')
ax2.set_ylabel('Uncertainty (σ)')
ax2.set_title('Filter Uncertainty Convergence')
ax2.legend()
ax2.grid(True, alpha=0.3)

# Plot 3: Kalman gain evolution
ax3 = axes[2]
for i, q in enumerate(q_values):
    ax3.plot(time, online_results[q]['gains'], colors[i], 
             linewidth=2, label=f'Q = {q}')
ax3.set_xlabel('Time')
ax3.set_ylabel('Kalman Gain (K)')
ax3.set_title('Kalman Gain Convergence')
ax3.legend()
ax3.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Parameter tuning guidance
print("\n" + "="*60)
print("PARAMETER TUNING GUIDE")
print("="*60)
print("""
Process Noise (Q):
  • Higher Q → Filter trusts observations more → More responsive
  • Lower Q  → Filter trusts model more → Smoother estimates
  • Typical approach: Set Q based on expected state variability

Observation Noise (R):
  • Higher R → Filter trusts model more → Smoother estimates  
  • Lower R  → Filter trusts observations more → More responsive
  • Can estimate from historical measurement variance

Ratio Q/R determines filter behavior:
  • High Q/R → Responsive, noisy estimates
  • Low Q/R  → Smooth, potentially lagged estimates

Practical Tips:
  1. Start with R from historical data variance
  2. Tune Q to get desired smoothness
  3. Use cross-validation on held-out data
  4. Consider adaptive methods for non-stationary data
""")

## 10. Advanced: Local Level Model with Trend (Level + Slope)

A more sophisticated model that captures both level and trend dynamics.